In [1]:
import sys, os
PATH = os.path.dirname(os.path.abspath(os.curdir))
if PATH not in sys.path:
    sys.path.insert(0, PATH)

import numpy as np
import scipy as sp
import torch
import h5py

import src.Optimizer as opt
import src.simulation_setup as setup
import hierarchical_algorithm as ha



In [2]:
##* create simulator 

buffer_train = "Experimental_data_CO_O_merged_train.hdf5"

const_dict, sim = setup.create_common_simulator(PATH, data_file=buffer_train)

Data Buffer:  /Users/joseafonso/Desktop/PlasmaDM/Buffer_Data/Experimental_data_CO_O_merged_train.hdf5
  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_35 - 0.02*CO_F*O_S*r_40 - 0.02*CO_F*Odb_S*r_61 - 0.02*CO_F*Vdb_S*r_60 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_36*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_36*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_39 - CO_S*r_37 - CO_S*r_43 - CO_S*r_44 - CO_S*r_45 - CO_S*r_46 + r_32*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0)
  d[COdb_S]/dt = CO_F*Vdb_S*r_60 - COdb_S*O_F*r_62 - COdb_S*r_54 - COdb_S*r_55 - COdb_S*r_56 - COdb_S*r_57 - COdb_S*r_59 + Vdb_S*r_49
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_35 - 0.02*CO_S*O_F*r_39 - 0.02*COdb_S*O_F*r_62 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_

In [3]:

# 2. Define Parameters and Bounds
lower_bounds_dict = {
    'A_d': 1e-8, 'B_d': 1e-8, 'E_d': 0.0, 
    'SF_30': 1e-5, 'SF_31': 1e-5, 'SF_32': 1e-5, 'SF_33': 1e-5, 'SF_34': 1e-5, 'SF_35': 1e-5, 'SF_36': 1e-5, 'SF_37': 1e-5, 'SF_38': 1e-5, 'SF_39': 1e-5,
    'SF_49': 1e-5, 'SF_50': 1e-5, 'SF_51': 1e-5, 'SF_52': 1e-5, 'SF_53': 1e-5, 'SF_54': 1e-5, 'SF_55': 1e-5, 'SF_56': 1e-5, 'SF_57': 1e-5, 'SF_58': 1e-5, 'SF_59': 1e-5, 'SF_60': 1e-5, 'SF_61': 1e-5, 'SF_62': 1e-5,
    'Emin': 1.0, 'Ealpha': 2000
}

upper_bounds_dict = {
    'A_d': 5e-1, 'B_d': 1e-2, 'E_d': 30.0, 
    'SF_30': 1.0, 'SF_31': 1.0, 'SF_32': 1.0,  'SF_33': 1.0, 'SF_34': 1.0, 'SF_35': 1.0, 'SF_36': 1.0, 'SF_37': 1.0, 'SF_38': 1.0, 'SF_39': 1.0,
    'SF_49': 1.0, 'SF_50': 1.0, 'SF_51': 1.0, 'SF_52': 1.0, 'SF_53': 1.0, 'SF_54': 1.0, 'SF_55': 1.0, 'SF_56': 1.0, 'SF_57': 1.0, 'SF_58': 1.0, 'SF_59': 1.0, 'SF_60': 1.0, 'SF_61': 1.0, 'SF_62': 1.0,
    'Emin': 5.0, 'Ealpha': 5000
}

params_default_dict = {
    'A_d': 0.02634, 'B_d': 7.67e-4, 'E_d': 10.75, 
    'SF_30': 1.0, 'SF_31': 1.0, 'SF_32': 1e-2,  'SF_33': 1e-1, 'SF_34': 1e-1, 'SF_35': 1e-2, 'SF_36': 1e-1, 'SF_37': 1e-1, 'SF_38': 1e-1, 'SF_39': 1e-1,
    'SF_49': 1e-2, 'SF_50': 1.0, 'SF_51': 1.0, 'SF_52': 1.0, 'SF_53': 1e-1, 'SF_54': 1e-1, 'SF_55': 1.0, 'SF_56': 1.0, 'SF_57': 1.0, 'SF_58': 1e-1, 'SF_59': 1e-1, 'SF_60': 1e-2, 'SF_61': 1e-1, 'SF_62': 1e-1,
    'Emin': 3.4, 'Ealpha': 3000
}

lower_bounds = np.array(list(lower_bounds_dict.values()))
upper_bounds = np.array(list(upper_bounds_dict.values()))
params_default_init = np.array(list(params_default_dict.values()))
params_default_norm = (params_default_init - lower_bounds) * np.reciprocal(upper_bounds - lower_bounds)


def func_optimization(params_input, flag='numpy'):
    
    ##! normalize variables
    params = [0] * len(params_input)
    for idx, param in enumerate(params_input):
        params[idx] = lower_bounds[idx] + (upper_bounds[idx] - lower_bounds[idx]) * param
    
    A_d, B_d, E_d = params[0:3]
    SF_30, SF_31, SF_32, SF_33, SF_34, SF_35, SF_36, SF_37, SF_38, SF_39 = params[3:13]
    SF_49, SF_50, SF_51, SF_52, SF_53, SF_54, SF_55, SF_56, SF_57, SF_58, SF_59, SF_60, SF_61, SF_62 = params[13:27]
    Emin, Ealpha = params[27:]
    
    if flag=='numpy':
        nu_d_mod = lambda T: 1e15 * (A_d + B_d * np.exp(E_d/(const_dict['R'] * T)))
    elif flag=='torch':
        nu_d_mod = lambda T: 1e15 * (A_d + B_d * torch.exp(E_d/(const_dict['R'] * T)))
    else:
        raise ValueError(f"{flag} does not exist")
    
    dict_mod_vec = [
    {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 16, "rate": None, "model_dict": {"Emin": Emin}},
    {"id": 18, "rate": None, "model_dict": {"Emin": Emin}},
    
    {"id": 31, "rate": None, "model_dict": {"SF": SF_31, "nu_d": nu_d_mod}},
    
    {"id": 30, "rate": None, "model_dict": {"SF": SF_30}},
    {"id": 32, "rate": None, "model_dict": {"SF": SF_32}},
    {"id": 33, "rate": None, "model_dict": {"SF": SF_33}},
    {"id": 34, "rate": None, "model_dict": {"SF": SF_34}},
    
    {"id": 35, "rate": None, "model_dict": {"SF": SF_35}},
    {"id": 36, "rate": None, "model_dict": {"SF": SF_36}},
    {"id": 37, "rate": None, "model_dict": {"SF": SF_37}},
    {"id": 38, "rate": None, "model_dict": {"SF": SF_38}},
    {"id": 39, "rate": None, "model_dict": {"SF": SF_39}},
    
    {"id": 44, "rate": None, "model_dict": {"Emin": Emin}},
    
    {"id": 49, "rate": None, "model_dict": {"SF": SF_49}},
    {"id": 50, "rate": None, "model_dict": {"SF": SF_50, "Ealpha": Ealpha}},
    {"id": 51, "rate": None, "model_dict": {"SF": SF_51, "Ealpha": Ealpha}},
    {"id": 52, "rate": None, "model_dict": {"SF": SF_52, "Ealpha": Ealpha}},
    {"id": 53, "rate": None, "model_dict": {"SF": SF_53, "Ealpha": Ealpha}},
    {"id": 54, "rate": None, "model_dict": {"SF": SF_54, "Ealpha": Ealpha}},
    {"id": 55, "rate": None, "model_dict": {"SF": SF_55, "Ealpha": Ealpha}},
    {"id": 56, "rate": None, "model_dict": {"SF": SF_56, "Ealpha": Ealpha}},
    {"id": 57, "rate": None, "model_dict": {"SF": SF_57, "Ealpha": Ealpha}},
    {"id": 58, "rate": None, "model_dict": {"SF": SF_58, "Ealpha": Ealpha}},
    {"id": 59, "rate": None, "model_dict": {"SF": SF_59, "Ealpha": Ealpha}},
    {"id": 60, "rate": None, "model_dict": {"SF": SF_60}},
    {"id": 61, "rate": None, "model_dict": {"SF": SF_61}},
    {"id": 62, "rate": None, "model_dict": {"SF": SF_62}}
    ]
    
    return dict_mod_vec

def loss_function(exp, teo, flag='numpy'):
    func = ((teo-exp)**2)/(exp**2)
    if flag == 'numpy':
        return np.mean(func)
    elif flag == 'torch':
        return torch.mean(func)
    else:
        raise ValueError(f"{flag} does not exist")


# 4. Instantiate and Run Optimizer
optimizer = opt.Optimizer(sim, 
                        lambda params: func_optimization(params, 'numpy'), 
                        lambda exp, teo: loss_function(exp, teo, 'numpy')
                        )

In [4]:


max_iters_tot = 1300
filename_results = "results/results_hier3_train_V4.h5"

pipeline = [
    ('decompose_stochastic', {'percent_info': 0.95, 'k_samples': 5}),
    ('optimize_stiff', {'max_iter': 70, 'bound_range': 1.1}),
    ('realign_sloppy', {}),
    ('optimize_sloppy', {'max_iter': 130}),
    ('check_convergence', {'tol_s': 1e-4})
]

# Instantiate
opt_hier = ha.HierarchicalOptimizer(optimizer, params_default_norm, pipeline=pipeline)
opt_hier.run(max_cycles=5, max_iter=max_iters_tot)


Starting Pipeline with 5 steps...

==================== CYCLE 1 ====================


loss:  783.7419346722897 iter:  1

[Step] Stochastic Decomposition (k=5)
loss:  783.7419346722897 iter:  2
loss:  783.7210654262981 iter:  3
loss:  783.8184680131944 iter:  4
loss:  783.8086440142472 iter:  5
loss:  783.7260913357984 iter:  6
loss:  783.6433184424724 iter:  7
  -> Split: Stiff=1, Sloppy=3

[Step] Stiff Opt (Powell) Dim:1
loss:  783.7419346722897 iter:  8
loss:  1.5930682691619262 iter:  9
loss:  13718.03030654385 iter:  10
loss:  0.6883630303689127 iter:  11
loss:  0.3622910451370119 iter:  12
loss:  0.5000369386432103 iter:  13


loss:  0.19958493966863575 iter:  14
loss:  0.25158701437756836 iter:  15
loss:  0.19715606215514833 iter:  16
loss:  0.1919138514351405 iter:  17
loss:  0.19675512523808275 iter:  18
loss:  0.19155420652506971 iter:  19
loss:  0.19154250931942382 iter:  20
loss:  0.19154154518678798 iter:  21
loss:  0.19154158488294515 iter:  22
loss:  0.19154165199543974 iter:  23
loss:  0.763275928372718 iter:  24
loss:  13718.030306263307 iter:  25
loss:  1.5930682691636506 iter:  26
loss:  0.6883630303618661 iter:  27
loss:  0.3622910451365202 iter:  28
loss:  0.5000369386450617 iter:  29


loss:  0.19958493966886334 iter:  30
loss:  0.2515870143714424 iter:  31
loss:  0.1971560621537828 iter:  32
loss:  0.19191385143118195 iter:  33
loss:  0.19675512608308926 iter:  34
loss:  0.19155420654902444 iter:  35
loss:  0.19154250931939978 iter:  36
loss:  0.19154154518391334 iter:  37
loss:  0.19154154236860035 iter:  38
loss:  0.19154155719449178 iter:  39
loss:  13718.030306284212 iter:  40
loss:  1.5930682691607072 iter:  41
loss:  0.6883630303618661 iter:  42
loss:  0.36229104513698124 iter:  43
loss:  0.5000369386456508 iter:  44


loss:  0.1995849396686251 iter:  45
loss:  0.25158701437167397 iter:  46
loss:  0.19715606215211637 iter:  47
loss:  0.19191385143283657 iter:  48
loss:  0.1967551260840976 iter:  49
loss:  0.19155420654983604 iter:  50
loss:  0.1915425093237439 iter:  51
loss:  0.19154154518856012 iter:  52
loss:  0.1915415413519198 iter:  53
loss:  0.19154154134732448 iter:  54
loss:  0.19154154134679877 iter:  55
loss:  0.19154154134896448 iter:  56
loss:  0.19154154134787135 iter:  57
loss:  0.19154154135081905 iter:  58
loss:  0.1915415413465164 iter:  59
loss:  0.1915415413544479 iter:  60
loss:  0.19154154134401158 iter:  61
loss:  0.19154154134637263 iter:  62
loss:  0.19154154134819856 iter:  63
  -> Loss: 0.191542 (Bounded)

[Step] Realigning Sloppy Space
loss:  0.19154154134401158 iter:  64
loss:  0.19154296649511465 iter:  65
loss:  0.19154086829320285 iter:  66
loss:  0.19154056160955232 iter:  67
  -> Realigned. Top Eigenvals: [272.40339699  14.05495776   4.30916849]

[Step] Sloppy Opt (N

...

loss:  0.09544329223208692 iter:  180
loss:  0.09134999501212822 iter:  181
loss:  0.09070490463169541 iter:  182
loss:  0.10597635521323517 iter:  183
loss:  0.09323740875265894 iter:  184
loss:  0.08920035247870531 iter:  185
loss:  0.08678349308152714 iter:  186
loss:  0.09090773576238698 iter:  187
loss:  0.11025982042937232 iter:  188
loss:  0.09016544481122793 iter:  189
loss:  0.09121340412867761 iter:  190
loss:  0.08885281849265406 iter:  191
loss:  0.08736686439339762 iter:  192
loss:  0.08587350302857351 iter:  193
loss:  0.08907440361719562 iter:  194
loss:  0.08711492555147121 iter:  195
loss:  0.0871651215742441 iter:  196
loss:  0.08600995968468512 iter:  197
  -> Max evals reached.
  -> Loss: 0.085874

==================== CYCLE 2 ====================

[Step] Stochastic Decomposition (k=5)
loss:  0.08587350302857351 iter:  198
loss:  0.08588570012004197 iter:  199
loss:  0.08587628954921003 iter:  200
loss:  0.08586970998091237 iter:  201
loss:  0.08587896372728976 iter

loss:  0.27383400593274454 iter:  209
loss:  0.18443428941279597 iter:  210
loss:  0.4734379291298124 iter:  211
loss:  0.11444682671189553 iter:  212
loss:  0.10040313853752683 iter:  213
loss:  0.15251231889819375 iter:  214
loss:  0.1708822537273791 iter:  215
loss:  0.09145669315133223 iter:  216
loss:  0.08916517565372885 iter:  217
loss:  0.091623674738444 iter:  218
loss:  0.08911493876887522 iter:  219
loss:  0.08902232255857061 iter:  220
loss:  0.08902131749036003 iter:  221
loss:  0.08902404835427426 iter:  222
  -> Loss: 0.089021 (Bounded)

[Step] Realigning Sloppy Space
loss:  0.08902131749036003 iter:  223
loss:  0.08901039087946329 iter:  224
loss:  0.08902489247017378 iter:  225
loss:  0.08902128955465657 iter:  226
  -> Realigned. Top Eigenvals: [8537.38877661 1063.51576627   14.55169019]

[Step] Sloppy Opt (Nelder-Mead) Dim:3
loss:  0.08902131749036003 iter:  227
loss:  0.08929821438989363 iter:  228
loss:  0.08910771115740061 iter:  229


loss:  0.08902186494061176 iter:  230
loss:  0.08880942320238232 iter:  231
loss:  0.08857398106940922 iter:  232
loss:  0.08863351606285198 iter:  233
loss:  0.08846892730524974 iter:  234
loss:  0.08820413546173816 iter:  235


loss:  0.08794781953405742 iter:  236
loss:  0.08745687064530876 iter:  237
loss:  0.08753514330355826 iter:  238
loss:  0.08695548967799517 iter:  239
loss:  0.0862616657929107 iter:  240
loss:  0.08608225822636818 iter:  241
loss:  0.08526173887126104 iter:  242
loss:  0.08519775372684339 iter:  243
loss:  0.08435889435192727 iter:  244
loss:  0.08382759324380884 iter:  245
loss:  0.0832061633321387 iter:  246
loss:  0.08318308129504576 iter:  247
loss:  0.08374235538388672 iter:  248
loss:  0.08330839909685345 iter:  249
loss:  0.08478418540856725 iter:  250
loss:  0.08340203177362662 iter:  251
loss:  0.08358047710344832 iter:  252
loss:  0.08318732240457141 iter:  253
loss:  0.08391985557932055 iter:  254
loss:  0.08314251772973275 iter:  255
loss:  0.083131357671017 iter:  256
loss:  0.0831576054529207 iter:  257
loss:  0.08325796134711455 iter:  258
loss:  0.0831445535816303 iter:  259
loss:  0.08311157967907175 iter:  260
loss:  0.08312985528717746 iter:  261
loss:  0.083217875

loss:  0.14517886273938185 iter:  349
loss:  0.1394922814985916 iter:  350
loss:  0.12036123103020571 iter:  351
loss:  0.10536323722733627 iter:  352
loss:  0.08508469233179229 iter:  353
loss:  0.11322752072634593 iter:  354
loss:  0.16627894621384015 iter:  355
loss:  0.0960538938456513 iter:  356
loss:  0.08403380745850286 iter:  357
loss:  0.08353902194017243 iter:  358
loss:  0.08344627324993563 iter:  359
loss:  0.08342660065444864 iter:  360
loss:  0.0834283835476994 iter:  361
loss:  0.08343051144394198 iter:  362
  -> Loss: 0.083427 (Bounded)

[Step] Realigning Sloppy Space
loss:  0.08342660065444864 iter:  363
loss:  0.08342590771186605 iter:  364
loss:  0.08342790363809388 iter:  365
loss:  0.08342683251540393 iter:  366
loss:  0.08342673943973536 iter:  367
  -> Realigned. Top Eigenvals: [1.16376886e+04 7.49911753e+01 7.45314890e+00]

[Step] Sloppy Opt (Nelder-Mead) Dim:4
loss:  0.08342660065444864 iter:  368
loss:  0.0834482914258429 iter:  369
loss:  0.08345936127187208 

In [5]:
with h5py.File(filename_results, "w") as f:
    
    f.create_dataset("best_loss", data=opt_hier.history['best_loss'])
    f.create_dataset("iters", data=opt_hier.history['iters'])
    f.create_dataset("best_params", data=opt_hier.history['best_params'])
    
    f.create_dataset("best_params_end", data=opt_hier.phi)